In [1]:
import pandas as pd
import psycopg2
import getpass
import requests
import time

In [2]:
def query_database(sql_query, user):
    '''
    This function take say SQL query and queries the casestdysu23t05 postgresql database.
    It returns a dataframe containing the result of that query.
    
    Args:
        sql_query: Str
            Contains the SQL query you want to query the database with.
        user: Str
    
    Returns:
        pandas.DataFrame
            Contains result of your query.
    '''
    
    sql_query = sql_query
    database = "casestdysu23t05"
    user     = 'smoc6k'
    password = getpass.getpass("Enter password: ")

    connection = psycopg2.connect(
        database = database,
        user = user,
        host = 'pgsql.dsa.lan',
        password = password)
    
    df = pd.read_sql_query(sql_query, connection)
    connection.close()
    return df



In [4]:
schema = 'raw'
table = 'jobs'
username = 'smoc6k'

sql_query = f'SELECT * FROM {schema}.{table} LIMIT 500'
df = query_database(sql_query, username)


In [30]:
df.count()

job_id              500
benefits            351
company_name        500
date_posted         352
description         500
items                17
location            500
posted_at           352
qualifications      468
responsibilities    463
salary               55
schedule_type       499
search_term         500
title               500
via                 500
work_from_home       56
date_scraped        500
dtype: int64

In [5]:
# Remove duplicate job postings based on description column.
df = df.drop_duplicates(subset='description')

In [6]:
df.count()

job_id              427
benefits            263
company_name        427
date_posted         306
description         427
items                35
location            427
posted_at           306
qualifications      375
responsibilities    370
salary               64
schedule_type       425
search_term         427
title               427
via                 427
work_from_home       57
date_scraped        427
dtype: int64

In [7]:
# Token count function to be used across dataframe columns.
def count_tokens(description):
    tokens = description.split()
    return len(tokens)

df['token_count'] = df['description'].apply(count_tokens)

df[['description', 'token_count']]

,description,token_count
0,Overview\nWhat to Expect\nWe are seeking a hig...,661
1,Summary\n\nApple’s Technology Development Grou...,513
2,What To Expect\nThe Quality Team at Gigafactor...,514
3,Employee Applicant Privacy Notice\n\nWho we ar...,758
4,Snap Inc is a technology company. We believe t...,837
...,...,...
494,Position Summary...\n\nWhat you'll do...\n\nAs...,1224
496,We have partnered with our client in their sea...,417
497,If you are unable to complete this application...,634
498,Additional Location(s): US-MN-Arden Hills; US-...,784


In [8]:
df.token_count.mean()

591.5035128805621

In [9]:
# Clean the dataframe description newline characters.
df_clean = df.copy()
df_clean['description'] = df_clean['description'].str.replace(r'\\n', '\n', regex=True)

In [10]:
print(df_clean['description'][3])

Employee Applicant Privacy Notice

Who we are...

Shape a brighter financial future with us.

Together with our members, we’re changing the way people think about and interact with personal finance.

We’re a next-generation fintech company using innovative, mobile-first technology to help our millions of members reach their goals. The industry is going through an unprecedented transformation, and we’re at the forefront. We’re proud to come to work every day knowing that what we do has a direct impact on people’s lives, with our core values guiding us every step of the way. Join us to invest in yourself, your career, and the financial world.

The Role

We are seeking a Staff Data Scientist who will help shape our Credit Card business strategy and product development decisions through data-driven insights. This is an exciting role for someone to make a direct impact on new product strategy and revenue of SoFi. Success in this role hinges on your technical aptitude, quantitative abilities

In [15]:
# OpenAI API key for connecting to service.
api_key = input("Please enter your OpenAI API key: ")

In [16]:
# Subset of cleaned description dataframe for use in the API.
df_subset = df_clean.head(10).copy()

# OPENAI CALL HERE


In [44]:
import json

def parse_testdata(testdata):
    message_content = testdata['choices'][0]['message']['content']
    try:
        parsed_content = json.loads(message_content)
    except json.JSONDecodeError:
        parsed_content = {"error": "Invalid JSON format"}
    return parsed_content

df_subset['industry2'] = df_subset['industry'].apply(parse_testdata)
df_subset.head()


,job_id,benefits,company_name,date_posted,description,items,location,posted_at,qualifications,responsibilities,...,via,work_from_home,date_scraped,token_count,testdata,newdata,new_min,new_max,industry,industry2
0,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCwgUmVjcn...,"Along with competitive pay, as a full-time Tes...",Tesla,2023-06-10,Overview\nWhat to Expect\nWe are seeking a hig...,None,"Palo Alto, CA",1 day ago,The ideal candidate will possess a deep unders...,"As a key member of our team, this person will ...",...,LinkedIn,NaN,2023-06-11,661,{'id': 'chatcmpl-8utwj2avmvn54OSEq3CU4HqWEFvE4...,"{'industry': 'Tech', 'minimum_salary': '$80,00...","$80,000","$258,000",{'id': 'chatcmpl-8uuYoykuiNj6dMSFZbroy0VUxL97L...,{'error': 'Invalid JSON format'}
1,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgRW5naW5lZX...,"Pay & Benefits\nAt Apple, base pay is one part...",Apple,2023-06-07,Summary\n\nApple’s Technology Development Grou...,None,"Cupertino, CA",4 days ago,Strong analytical skills\nExcellent written an...,Apple’s Technology Development Group is lookin...,...,LinkedIn,NaN,2023-06-11,513,{'id': 'chatcmpl-8utwkvAor2jEwhMpMLsYGGzSh4f4A...,{'error': 'Invalid JSON format'},None,None,{'id': 'chatcmpl-8uuYpUKkfSDbkCdBn8Dxn6L1xE2Mm...,"{'Industry': 'Technology', 'Minimum Pay': 121000}"
2,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAvIFF1YW...,"Along with competitive pay, as a full-time Tes...",Tesla,2023-06-08,What To Expect\nThe Quality Team at Gigafactor...,None,"Sparks, NV",3 days ago,Extensive experience writing software with Pyt...,This position will support a variety of projec...,...,LinkedIn,NaN,2023-06-11,514,{'id': 'chatcmpl-8utwmla6zpmRX0uLTjEOIQOOurQP9...,"{'industry': 'Tech', 'minimum_salary': 'NA', '...",NA,NA,{'id': 'chatcmpl-8uuYqTp41Pxe6L1TeSUlbRfdy7UDd...,{'min_pay': None}
3,eyJqb2JfdGl0bGUiOiJTdGFmZiBEYXRhIFNjaWVudGlzdC...,Compensation And Benefits\nThe base pay range ...,SoFi,2023-06-10,Employee Applicant Privacy Notice\n\nWho we ar...,None,"Salt Lake City, UT",1 day ago,"M.B.A, B.S. or M.S. in Statistics, Economics, ...",We are seeking a Staff Data Scientist who will...,...,LinkedIn,NaN,2023-06-11,758,{'id': 'chatcmpl-8utwnNzkM9EBo5MwggPSnc8RKo7zg...,"{'industry': 'Finance', 'minimum_salary': '$12...","$120,000","$190,000",{'id': 'chatcmpl-8uuYqs2Q1OsEg0zGTjHyky2O9ygek...,{'minimum_pay': None}
4,eyJqb2JfdGl0bGUiOiJFbmdpbmVlcmluZyBEYXRhIFNjaW...,"In the United States, work locations are assig...",Snap Inc.,2023-06-05,Snap Inc is a technology company. We believe t...,None,"Palo Alto, CA",6 days ago,Strong statistical knowledge\nAbility to commu...,Build state-of-the-art statistical models to s...,...,LinkedIn,NaN,2023-06-11,837,{'id': 'chatcmpl-8utwpsrZwN34G7gkJAwZmF0vP06ge...,"{'industry': 'Tech', 'minimum_salary': '$196,0...","$196,000","$335,000",{'id': 'chatcmpl-8uuYrsB8hZl0pdwj4MN9kJbH7Zcji...,196000


In [43]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

def extract_info(description):
    data = {
        "model": "gpt-3.5-turbo",  
        "messages": [{"role": "system","content":"You are being provided with a job description. You will be asked to extract several features from the job description, if the job description contains the information."},
                     {"role": "user", "content": "'"+ description + "'"+ "In a single word, what industry is the company in? Pass in json format."},
                     {"role": "user", "content": "Now give the minimum pay that the job offers, if the information is available. Only pass numbers and in json format if the minimum pay is found. If the minimum pay is not found pass None and in json format."}],
        ""
        "max_tokens": 500
    }
    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=data)
    return response.json()

df_subset['industry'] = df_subset['description'].apply(lambda x: extract_info(x))

In [32]:
df_subset

,job_id,benefits,company_name,date_posted,description,items,location,posted_at,qualifications,responsibilities,...,title,via,work_from_home,date_scraped,token_count,testdata,newdata,new_min,new_max,industry
0,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCwgUmVjcn...,"Along with competitive pay, as a full-time Tes...",Tesla,2023-06-10,Overview\nWhat to Expect\nWe are seeking a hig...,None,"Palo Alto, CA",1 day ago,The ideal candidate will possess a deep unders...,"As a key member of our team, this person will ...",...,"Data Scientist, Recruiting Operations",LinkedIn,NaN,2023-06-11,661,{'id': 'chatcmpl-8utwj2avmvn54OSEq3CU4HqWEFvE4...,"{'industry': 'Tech', 'minimum_salary': '$80,00...","$80,000","$258,000",{'id': 'chatcmpl-8uuDcrCREkTtsOjgYrzMq5TQ2EapG...
1,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgRW5naW5lZX...,"Pay & Benefits\nAt Apple, base pay is one part...",Apple,2023-06-07,Summary\n\nApple’s Technology Development Grou...,None,"Cupertino, CA",4 days ago,Strong analytical skills\nExcellent written an...,Apple’s Technology Development Group is lookin...,...,Data Science Engineer,LinkedIn,NaN,2023-06-11,513,{'id': 'chatcmpl-8utwkvAor2jEwhMpMLsYGGzSh4f4A...,{'error': 'Invalid JSON format'},None,None,{'id': 'chatcmpl-8uuDco8vub2PgkT6Y7haN4W5OgjFU...
2,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAvIFF1YW...,"Along with competitive pay, as a full-time Tes...",Tesla,2023-06-08,What To Expect\nThe Quality Team at Gigafactor...,None,"Sparks, NV",3 days ago,Extensive experience writing software with Pyt...,This position will support a variety of projec...,...,Data Scientist / Quality Engineering Nevada,LinkedIn,NaN,2023-06-11,514,{'id': 'chatcmpl-8utwmla6zpmRX0uLTjEOIQOOurQP9...,"{'industry': 'Tech', 'minimum_salary': 'NA', '...",NA,NA,{'id': 'chatcmpl-8uuDdC4JqUMabosoHouvTGWsP4tKf...
3,eyJqb2JfdGl0bGUiOiJTdGFmZiBEYXRhIFNjaWVudGlzdC...,Compensation And Benefits\nThe base pay range ...,SoFi,2023-06-10,Employee Applicant Privacy Notice\n\nWho we ar...,None,"Salt Lake City, UT",1 day ago,"M.B.A, B.S. or M.S. in Statistics, Economics, ...",We are seeking a Staff Data Scientist who will...,...,"Staff Data Scientist, Credit Card",LinkedIn,NaN,2023-06-11,758,{'id': 'chatcmpl-8utwnNzkM9EBo5MwggPSnc8RKo7zg...,"{'industry': 'Finance', 'minimum_salary': '$12...","$120,000","$190,000",{'id': 'chatcmpl-8uuDddpkCvPkYvwRbFFvHPBFFElem...
4,eyJqb2JfdGl0bGUiOiJFbmdpbmVlcmluZyBEYXRhIFNjaW...,"In the United States, work locations are assig...",Snap Inc.,2023-06-05,Snap Inc is a technology company. We believe t...,None,"Palo Alto, CA",6 days ago,Strong statistical knowledge\nAbility to commu...,Build state-of-the-art statistical models to s...,...,"Engineering Data Scientist, Ad Ranking",LinkedIn,NaN,2023-06-11,837,{'id': 'chatcmpl-8utwpsrZwN34G7gkJAwZmF0vP06ge...,"{'industry': 'Tech', 'minimum_salary': '$196,0...","$196,000","$335,000",{'id': 'chatcmpl-8uuDePoZudsmT6VynRW2kuKpULIYY...
5,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAvIFF1YW...,"Along with competitive pay, as a full-time Tes...",Tesla,2023-06-08,What To Expect\nThe successful candidate for t...,None,"Lathrop, CA",3 days ago,The successful candidate for this position mus...,"Development of manufacturing requirements, qua...",...,Data Scientist / Quality Engineering Lathrop,LinkedIn,NaN,2023-06-11,604,{'id': 'chatcmpl-8utwqZlIuw6xxGWLQLBjOWuTkRqE4...,"{'industry': 'Tech', 'minimum salary': '$72,00...",None,None,{'id': 'chatcmpl-8uuDfJ0OygfCgUddECyTzBmLRHwIH...
6,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,Compensation may be adjusted depending on work...,Cloudflare,2023-06-08,"About Us\n\nAt Cloudflare, we have our eyes se...",None,"Phoenix, AZ",3 days ago,"M.S or Ph.D in Computer Science, Statistics, M...","Partner and align with business leaders, stake...",...,Data Scientist,LinkedIn,NaN,2023-06-11,1433,{'id': 'chatcmpl-8utwr6I44h8EVE3Oi0eeUDMbNTvj5...,"{'industry': 'Tech', 'minimum_salary': '$140,0...","$140,000","$202,000",{'id': 'chatcmpl-8uuDguM0yKKdAlZ8tUcRuEYDG6KFc...
7,eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0

In [17]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'
}

def extract_info(description):
    data = {
        "model": "gpt-3.5-turbo",  
        "messages": [{"role": "system", "content": "You are building a json file for each of the following job posts. Accuracy and attention to formatting is crucial."}, 
                     {"role": "user", "content": "'"+ description + "'"+ "You are finding key information in the document. We are looking for the following fields;one word description of the industry the company is in(simple answers: None, Finance, Tech, Retail, etc), minimum salary($xxx,xxx or NA - No other formats), maximum salary($xxx,xxx or NA - No other formats). Return this as a json."}],
        "max_tokens": 500
    }
    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=data)
    return response.json()

start_time = time.time()

df_subset['testdata'] = df_subset['description'].apply(lambda x: extract_info(x))

end_time = time.time()

print(f"Execution time: {end_time - start_time} seconds")

Execution time: 11.127667427062988 seconds


In [18]:
print(df_subset.testdata[1])

{'id': 'chatcmpl-8utwkvAor2jEwhMpMLsYGGzSh4f4A', 'object': 'chat.completion', 'created': 1708572118, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "industry": "Tech",\n  "minimum_salary": "$121,000",\n  "maximum_salary": "$230,000"\n}\n```'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 724, 'completion_tokens': 33, 'total_tokens': 757}, 'system_fingerprint': 'fp_cbdb91ce3f'}


In [20]:
import json

def parse_testdata(testdata):
    message_content = testdata['choices'][0]['message']['content']
    try:
        parsed_content = json.loads(message_content)
    except json.JSONDecodeError:
        parsed_content = {"error": "Invalid JSON format"}
    return parsed_content

df_subset['newdata'] = df_subset['testdata'].apply(parse_testdata)
df_subset.head()


,job_id,benefits,company_name,date_posted,description,items,location,posted_at,qualifications,responsibilities,salary,schedule_type,search_term,title,via,work_from_home,date_scraped,token_count,testdata,newdata
0,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCwgUmVjcn...,"Along with competitive pay, as a full-time Tes...",Tesla,2023-06-10,Overview\nWhat to Expect\nWe are seeking a hig...,None,"Palo Alto, CA",1 day ago,The ideal candidate will possess a deep unders...,"As a key member of our team, this person will ...",None,Full-time,Data Scientist,"Data Scientist, Recruiting Operations",LinkedIn,NaN,2023-06-11,661,{'id': 'chatcmpl-8utwj2avmvn54OSEq3CU4HqWEFvE4...,"{'industry': 'Tech', 'minimum_salary': '$80,00..."
1,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVuY2UgRW5naW5lZX...,"Pay & Benefits\nAt Apple, base pay is one part...",Apple,2023-06-07,Summary\n\nApple’s Technology Development Grou...,None,"Cupertino, CA",4 days ago,Strong analytical skills\nExcellent written an...,Apple’s Technology Development Group is lookin...,None,Full-time,Data Scientist,Data Science Engineer,LinkedIn,NaN,2023-06-11,513,{'id': 'chatcmpl-8utwkvAor2jEwhMpMLsYGGzSh4f4A...,{'error': 'Invalid JSON format'}
2,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAvIFF1YW...,"Along with competitive pay, as a full-time Tes...",Tesla,2023-06-08,What To Expect\nThe Quality Team at Gigafactor...,None,"Sparks, NV",3 days ago,Extensive experience writing software with Pyt...,This position will support a variety of projec...,None,Full-time,Data Scientist,Data Scientist / Quality Engineering Nevada,LinkedIn,NaN,2023-06-11,514,{'id': 'chatcmpl-8utwmla6zpmRX0uLTjEOIQOOurQP9...,"{'industry': 'Tech', 'minimum_salary': 'NA', '..."
3,eyJqb2JfdGl0bGUiOiJTdGFmZiBEYXRhIFNjaWVudGlzdC...,Compensation And Benefits\nThe base pay range ...,SoFi,2023-06-10,Employee Applicant Privacy Notice\n\nWho we ar...,None,"Salt Lake City, UT",1 day ago,"M.B.A, B.S. or M.S. in Statistics, Economics, ...",We are seeking a Staff Data Scientist who will...,162K–248K a year,Full-time,Data Scientist,"Staff Data Scientist, Credit Card",LinkedIn,NaN,2023-06-11,758,{'id': 'chatcmpl-8utwnNzkM9EBo5MwggPSnc8RKo7zg...,"{'industry': 'Finance', 'minimum_salary': '$12..."
4,eyJqb2JfdGl0bGUiOiJFbmdpbmVlcmluZyBEYXRhIFNjaW...,"In the United States, work locations are assig...",Snap Inc.,2023-06-05,Snap Inc is a technology company. We believe t...,None,"Palo Alto, CA",6 days ago,Strong statistical knowledge\nAbility to commu...,Build state-of-the-art statistical models to s...,None,Full-time,Data Scientist,"Engineering Data Scientist, Ad Ranking",LinkedIn,NaN,2023-06-11,837,{'id': 'chatcmpl-8utwpsrZwN34G7gkJAwZmF0vP06ge...,"{'industry': 'Tech', 'minimum_salary': '$196,0..."


In [21]:
# Function to extract minimum and maximum salary
def extract_min_salary(newdata):
    return newdata.get('minimum_salary', None)

def extract_max_salary(newdata):
    return newdata.get('maximum_salary', None)

# Apply these functions to extract and assign values to new_min and new_max columns
df_subset['new_min'] = df_subset['newdata'].apply(extract_min_salary)
df_subset['new_max'] = df_subset['newdata'].apply(extract_max_salary)

# Check the updated DataFrame
df_subset['newdata']


0    {'industry': 'Tech', 'minimum_salary': '$80,00...
1                     {'error': 'Invalid JSON format'}
2    {'industry': 'Tech', 'minimum_salary': 'NA', '...
3    {'industry': 'Finance', 'minimum_salary': '$12...
4    {'industry': 'Tech', 'minimum_salary': '$196,0...
5    {'industry': 'Tech', 'minimum salary': '$72,00...
6    {'industry': 'Tech', 'minimum_salary': '$140,0...
7    {'industry': 'Tech', 'minimum_salary': '$80,00...
8    {'industry': 'Finance', 'minimum_salary': '$12...
9    {'industry': 'Retail', 'minimum_salary': 'NA',...
Name: newdata, dtype: object

In [160]:
df1_subset.newdata[1]


{'industry': 'Retail', 'min_salary': 'NA', 'max_salary': 'NA'}

In [161]:
def format_row(row):
    formatted_text = (
        "-------------------------------------------------------\n"
        f"Column: {row['title']}\n"
        f"Column: {row['company_name']}\n\n"
        f"{row['description']}\n\n"
        f"Column MinSalary: {row['new_min']}\n"
        f"Column MaxSalary: {row['new_max']}\n"
        f"Received from Chatgpt3.5: {row['testdata']}\n"
    )
    return formatted_text

# Apply the formatting function to each row and concatenate the results
formatted_data = df1_subset.apply(format_row, axis=1).str.cat(sep='\n')

# Write to a text file
with open('test5.txt', 'w') as file:
    file.write(formatted_data)

print("Export completed. The file is saved as df1_subset_formatted.txt")


Export completed. The file is saved as df1_subset_formatted.txt


In [162]:
for _, row in df1_subset.iterrows():
    print(f'  {row.company_name} ----> {row.newdata.get("industry")}')
    print(f'{row.new_min}')
    print(f'{row.new_max}')
    print(f'{row.newdata.get("programming_technical_skills")}')
    print(f'-------------------')

  Etsy ----> Retail
$104,000
$136,000
None
-------------------
  Walmart ----> Retail
None
None
None
-------------------
  Glocomms ----> Finance
NA
NA
None
-------------------
  Transfix ----> Logistics
$89,250
$150,500
None
-------------------
  Obviously ----> Tech
NA
NA
None
-------------------
  SELF ID ----> Tech
NA
NA
None
-------------------
  MediaMath ----> Tech
$231,000
$300,000
None
-------------------
  E-Frontiers ----> Tech
$NA
$NA
None
-------------------
  Internal Revenue Service ----> Finance
NA
NA
None
-------------------
  Metropolitan Transportation Authority ----> Transportation
$65,971
$105,740
None
-------------------
  The D. E. Shaw Group ----> Finance
NA
NA
None
-------------------
  Google ----> Tech
$146,000
$220,000
None
-------------------
  Philo ----> Tech
$160,000
$210,000
None
-------------------
  SynergisticIT ----> Tech
NA
NA
None
-------------------
  SoFi ----> Finance
NA
NA
None
-------------------
  Mount Sinai Health Systems ----> Healthcare
